In [2]:
import pandas as pd

### Lectura del fichero CSV

In [5]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

train_data = data[data['Set'] == 'Train']
test_data = data[data['Set'] == 'Test']
